# Make a catalog of SEDs

In [1]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
import copy
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in /grp/hst/cdbs/grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:114: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /grp/hst/cdbs/mtab/*_tmg.fits.
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:121: UserWarning: No thermal tables found, no thermal calculations can be performed. No files 

Loading BokehJS ...

First we initialize an empty catalog.

In [2]:
# Make an SED catalog
cat = sed.SEDCatalog()

Now let's make a few SEDs

In [3]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [4]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    s2.make_sed()
    cat.add_SED(s2)

Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!


Then the aggregated results look like this:

In [5]:
cat.results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11249aba8>
2MASS 2618328860,6000.0,4000.0,629.25,39.6,1.5891833265354647,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.4271354764110654e+32,5.588076051088746e+31,-0.937,0.055,7.098,0.069,4.5,0.07,0.8896913720017506,0.0,3590.0,178.0,<SEDkit.sed.SED object at 0x11292a2e8>
2MASS 2618328861,6000.0,4000.0,521.28,27.17,1.9183651494507252,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.0382148981339667e+32,3.180305465424505e+31,-1.1,0.045,7.507,0.057,4.5,0.07,0.8896913720017506,0.0,3268.0,152.0,<SEDkit.sed.SED object at 0x112934128>
2MASS 2618328862,6000.0,4000.0,601.41,36.17,1.6627699556640692,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.0440605915773985e+32,4.879554480360618e+31,-0.976,0.052,7.196,0.066,4.5,0.07,0.8896913720017506,0.0,3510.0,171.0,<SEDkit.sed.SED object at 0x1124a30b8>
2MASS 2618328863,6000.0,4000.0,512.56,26.27,1.9510032516779738,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,2.9374182292124785e+32,3.0239477545572067e+31,-1.115,0.045,7.543,0.057,4.5,0.07,0.8896913720017506,0.0,3240.0,150.0,<SEDkit.sed.SED object at 0x1124b51d0>
2MASS 2618328864,6000.0,4000.0,633.41,40.12,1.5787522126224143,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.4858649511952065e+32,5.698675198057469e+31,-0.931,0.055,7.084,0.069,4.5,0.07,0.8896913720017506,0.0,3602.0,180.0,<SEDkit.sed.SED object at 0x1124c3ac8>
2MASS 2618328865,6000.0,4000.0,551.59,30.43,1.812939658125274,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.401802781350719e+32,3.7673303911447893e+31,-1.051,0.048,7.384,0.061,4.5,0.07,0.8896913720017506,0.0,3362.0,159.0,<SEDkit.sed.SED object at 0x112916860>
2MASS 2618328866,6000.0,4000.0,546.43,29.86,1.8300769599327296,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.338454284241592e+32,3.6624397763936157e+31,-1.059,0.048,7.404,0.06,4.5,0.07,0.8896913720017506,0.0,3346.0,158.0,<SEDkit.sed.SED object at 0x1129169e8>


We can also plot the results by passing the `plot` method two column names.

In [6]:
show(cat.plot('distance', 'Teff'))

We can then retrieve the SED for a particular source like so:

In [8]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()